In [1]:
import pandas as pd
from datetime import date
import time
pd.set_option('display.max_columns', None) # PAra que se vean todas las columnas de los df

In [2]:
df = pd.read_csv(r'scopus_bd_aumentada.csv', sep=",")

C:\Users\PC1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(3)

,Authors,Author(s)_ID,Authors_Rev,Author(s)_ID_Rev,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,Page_end,Page_count,Cited_by,DOI,Link,Affiliations,Authors_with_affiliations,Abstract,Author_Keywords,Index_Keywords,ISSN,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID
0,Vanzolini P.E.,6602373241,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,611,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Museu de Zoologia, Universidade de São Paulo, ...","Vanzolini, P.E., Museu de Zoologia, Universida...","Gymnodactylus amarali Barbour, 1925, was previ...",Lizards; Speciation; Systematics,animal; article; Brazil; classification; compa...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573
1,De Wagener A.L.R.,7004422261,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,623,NaN,6.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Departamento de Química, Pontifícia Universida...","De Wagener, A.L.R., Departamento de Química, P...",This paper addresses the limitations the scarc...,Coastal management; Developing countries; Envi...,article; developing country; environmental mon...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Mexias A.S.,11739855600,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319


In [4]:
df_authors = df[['Authors_Rev','Author(s)_ID_Rev']][:].drop_duplicates(subset='Author(s)_ID_Rev')

### Total Citaciones

In [5]:
df_temp=df[['Author(s)_ID_Rev', 'Cited_by']].groupby(by='Author(s)_ID_Rev').sum()
df_temp.reset_index(level=0, inplace=True)

In [6]:
df_authors = df_authors.merge(right=df_temp, how='inner', on='Author(s)_ID_Rev')

### Total Artículos publicados

In [7]:
df_temp=df[['Author(s)_ID_Rev', 'Title']].groupby(by='Author(s)_ID_Rev').count()
#Convierte el indice en otro campo
df_temp.reset_index(level=0, inplace=True)

In [8]:
df_authors = df_authors.merge(df_temp, how='inner', on ='Author(s)_ID_Rev')

### Máximo número de citaciones

In [9]:
df_temp=df[['Author(s)_ID_Rev', 'Cited_by']].groupby(by='Author(s)_ID_Rev').max()
df_temp.reset_index(level=0, inplace=True)

In [10]:
df_authors = df_authors.merge(right=df_temp, how='inner', on='Author(s)_ID_Rev')
df_authors.columns = ['Authors_Rev', 'Author(s)_ID_Rev', 'Total_Citation', 'Total_Title', 'Max_Citation']
df_authors['Max_Citation'].fillna(0, inplace=True)

### Número de Revistas

In [11]:
df_temp=df[['Author(s)_ID_Rev', 'Source_title']].groupby(by=['Author(s)_ID_Rev']).nunique()
df_temp = df_temp[['Source_title']]
df_temp.reset_index(level=0, inplace=True)
df_temp.columns = ['Author(s)_ID_Rev','Total_Journal']
df_temp.sort_values(by='Total_Journal', ascending = False, inplace= True)

In [12]:
df_authors = df_authors.merge(df_temp, how='inner', on ='Author(s)_ID_Rev')

In [13]:
año_actual=int(date.today().strftime("%Y"))
año_pasado=año_actual-1

In [14]:
df2_años=df[(df['Year']==año_actual) | (df['Year']==año_pasado) ]

### Total Publicaciones últimos dos años

In [15]:
df_temp=df2_años[['Author(s)_ID_Rev', 'Title']].groupby(by='Author(s)_ID_Rev').count()
df_temp.reset_index(level=0, inplace=True)
df_temp.columns = ['Author(s)_ID_Rev','Delta_publications']
df_authors = df_authors.merge(df_temp, how='left', on ='Author(s)_ID_Rev')

In [16]:
df_authors['Delta_publications'].fillna(0, inplace=True)


### Carrera Autor

In [17]:
df_temp = df[['Author(s)_ID_Rev','Year']].groupby(by = 'Author(s)_ID_Rev').min()
df_temp.reset_index(level=0, inplace=True)
df2_temp = df[['Author(s)_ID_Rev','Year']].groupby(by = 'Author(s)_ID_Rev').max()
df2_temp.reset_index(level=0, inplace=True)
df_temp = df_temp.merge(df2_temp, how='left', on='Author(s)_ID_Rev')
df_temp['autor_career'] =  df_temp['Year_y'] - df_temp['Year_x']
df_authors = df_authors.merge(df_temp[['Author(s)_ID_Rev','autor_career']], how='left', on ='Author(s)_ID_Rev')

### Citación Media por Revista

In [18]:
df_temp = df[['Source_title','Cited_by']].groupby(by='Source_title').mean().fillna(0)
df_temp.reset_index(level=0, inplace=True)

In [19]:
df2_temp = df[['Author(s)_ID_Rev','Source_title','Cited_by']].groupby(by=['Author(s)_ID_Rev','Source_title']).mean().fillna(0)
df2_temp.reset_index(level=['Author(s)_ID_Rev','Source_title'],inplace=True)
df2_temp =df2_temp.merge(df_temp, how='left', on='Source_title')
df2_temp = df2_temp[['Author(s)_ID_Rev','Cited_by_y']].groupby(by='Author(s)_ID_Rev').mean()
df2_temp.reset_index(level=0,inplace=True)
df2_temp.columns = ['Author(s)_ID_Rev','citation_journal']
df_authors = df_authors.merge(df2_temp, how='left', on='Author(s)_ID_Rev')

### Total Coautores ultimos dos años

#### Leer
* Identificación de Todos los autores
* para cada autor se revisan los papers que en los que aparece
* para cada paper de ese autor se revisan cuantas veces aparece bajo un nombre diferente al del autor superior
* si el autor que aparece no se ha añadido a la lista de coautores se hace.

# Esto se demora corriendo alrededor de 10 minutos
## puedes irte a tomar una bebida caliente mientras tanto
### es en serio :( :(

In [20]:
start = time.time()
#Línea de prueba para correr rápido
#lista_autores = df2_años['Author(s)_ID_Rev'][df2_años['Author(s)_ID_Rev']=='53866250100'].unique()
lista_autores = df2_años['Author(s)_ID_Rev'].unique()[:]
lista_coautores = []
for autor in lista_autores:
    df_paper = df2_años[['Author(s)_ID_Rev','Title']][df2_años['Author(s)_ID_Rev']==autor]
    lista_papers = df_paper['Title']
    coautores = []
    for paper in lista_papers:
        coautor = (df2_años[['Author(s)_ID_Rev','Title']][(df2_años['Title']==paper)&(df2_años['Author(s)_ID_Rev']!=autor)]
                   ['Author(s)_ID_Rev'])
        if len(coautor)==0:
            continue
        else:
            coautor = (df2_años[['Author(s)_ID_Rev','Title']][(df2_años['Title']==paper)&(df2_años['Author(s)_ID_Rev']!=autor)]
                   ['Author(s)_ID_Rev'])
        for c in coautor:
            if c is not coautores:
                coautores.append(c)
    del(df_paper)
    lista_coautores.append([autor,len(coautores)])
end = time.time()
#print((end - start)/60)

In [21]:
#Tiempo de ejecución
(end -start)/60

4.561218750476837

### lista_coautores

In [23]:
df2_temp = pd.DataFrame(lista_coautores, columns=['Author(s)_ID_Rev','Delta_coautores'])

In [24]:
df_authors = df_authors.merge(df2_temp, how='left', on='Author(s)_ID_Rev').sort_values(by='Delta_publications', ascending = False)

# Indicadores Papers

* Citaciones por Artículos
* años del artículos
* Cantidad de coautures

In [25]:
df_papers =df[['Title','Cited_by']].drop_duplicates(subset='Title').fillna(0)
df_temp = df[['Title','Year']].drop_duplicates(subset='Title').fillna(0)
df_temp['Year'] = año_actual - df_temp['Year']
df_papers = df_papers.merge(df_temp, how='left', on = 'Title')
df_temp = df[['Title', 'Author(s)_ID_Rev']].fillna(0).groupby(by='Title').count()
df_temp.reset_index(level=0, inplace= True)
df_papers = df_papers.merge(df_temp, how='left', on = 'Title')
df_papers.columns = ['Title', 'Cited_by', 'Years', 'cant_coautores']

### Citaciones por revista

In [26]:
#Revistas y Citaciones
df_temp = df[['Source_title','Cited_by']].groupby(by='Source_title').mean().fillna(0)
df_temp.reset_index(level=0, inplace=True)

In [27]:
df2_temp = df[['Title','Source_title','Cited_by']].groupby(by=['Title','Source_title']).mean().fillna(0)
df2_temp.reset_index(level=['Title','Source_title'],inplace=True)
df2_temp =df2_temp.merge(df_temp, how='left', on='Source_title')
df2_temp = df2_temp[['Title','Cited_by_y']].groupby(by='Title').mean()
df2_temp.reset_index(level=0,inplace=True)
df2_temp.columns = ['Title','citation_journal']
df_papers = df_papers.merge(df2_temp, how='left', on='Title')

In [28]:
df_authors.to_csv('indicadores_autores.csv')
df_papers.to_csv('indicadores_papers.csv')

### Cosolidacion de los datos

In [30]:
df_papers.head()

,Title,Cited_by,Years,cant_coautores,citation_journal
0,"On Gymnodactylus amarali Barbour, 1925, with t...",7.0,14,1,9.146637
1,Constraints to the implementation of effective...,6.0,14,1,9.146637
2,Geochemical modeling of gold precipitation con...,7.0,14,7,9.146637
3,The specificity of interactions between protei...,87.0,14,1,9.146637
4,On the retrieval of significant wave heights f...,1.0,14,1,9.146637


In [32]:
df_authors.head()

,Authors_Rev,Author(s)_ID_Rev,Total_Citation,Total_Title,Max_Citation,Total_Journal,Delta_publications,autor_career,citation_journal,Delta_coautores
100456,Guarda T.,53866250100,1.0,20,1.0,1,20.0,1,4.797178,60.0
926,[No author name available],[No author id available,53.0,390,8.0,42,18.0,46,5.077336,0.0
24466,Alsaedi A.,55664358300,223.0,30,34.0,2,18.0,6,9.063147,55.0
24464,Hayat T.,8856998000,379.0,37,83.0,3,18.0,8,10.197079,55.0
63010,Khan M.,55579497400,132.0,22,33.0,1,17.0,3,6.240342,47.0


In [31]:
df.head()

,Authors,Author(s)_ID,Authors_Rev,Author(s)_ID_Rev,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,Page_end,Page_count,Cited_by,DOI,Link,Affiliations,Authors_with_affiliations,Abstract,Author_Keywords,Index_Keywords,ISSN,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID
0,Vanzolini P.E.,6602373241,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,611,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Museu de Zoologia, Universidade de São Paulo, ...","Vanzolini, P.E., Museu de Zoologia, Universida...","Gymnodactylus amarali Barbour, 1925, was previ...",Lizards; Speciation; Systematics,animal; article; Brazil; classification; compa...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573
1,De Wagener A.L.R.,7004422261,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,623,NaN,6.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Departamento de Química, Pontifícia Universida...","De Wagener, A.L.R., Departamento de Química, P...",This paper addresses the limitations the scarc...,Coastal management; Developing countries; Envi...,article; developing country; environmental mon...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Mexias A.S.,11739855600,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319
3,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Berger G.,7202349172,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319
4,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Gomes M.E.B.,11738879800,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319


In [33]:
# Reordenamiento del indice del df autores
df_authors.sort_index(inplace=True)

In [34]:
df_authors.head()

,Authors_Rev,Author(s)_ID_Rev,Total_Citation,Total_Title,Max_Citation,Total_Journal,Delta_publications,autor_career,citation_journal,Delta_coautores
0,Vanzolini P.E.,6602373241,62.0,5,17.0,1,0.0,3,9.146637,NaN
1,De Wagener A.L.R.,7004422261,6.0,1,6.0,1,0.0,0,9.146637,NaN
2,Mexias A.S.,11739855600,11.0,2,7.0,1,0.0,2,9.146637,NaN
3,Berger G.,7202349172,7.0,1,7.0,1,0.0,0,9.146637,NaN
4,Gomes M.E.B.,11738879800,11.0,2,7.0,1,0.0,2,9.146637,NaN


In [35]:
# Número de id de autores unicos
len(df_authors['Author(s)_ID_Rev'].unique())

113948

In [36]:
# Validar si la columna no es constante
df_authors['Total_Journal'].unique()

array([ 1,  2,  3,  4, 42,  5,  6,  8,  7, 11], dtype=int64)

In [37]:
# Dimensiones del df resultante
df_authors.shape

(113948, 10)

In [38]:
# Dimensiones del df original
df.shape

(177350, 30)

In [39]:
df_papers.head()

,Title,Cited_by,Years,cant_coautores,citation_journal
0,"On Gymnodactylus amarali Barbour, 1925, with t...",7.0,14,1,9.146637
1,Constraints to the implementation of effective...,6.0,14,1,9.146637
2,Geochemical modeling of gold precipitation con...,7.0,14,7,9.146637
3,The specificity of interactions between protei...,87.0,14,1,9.146637
4,On the retrieval of significant wave heights f...,1.0,14,1,9.146637


In [40]:
# Creacion de un df vacio que se utilizará para consolidar toda la información
df_consol = pd.DataFrame()

In [41]:
# Union del df inicial con el df de papers
df_consol = pd.concat([df, df_papers], axis=1, sort=False)#df.merge(df_papers, left_on='Title', right_on='Title')

In [42]:
# Revisión df consolidado
df_consol.head(3)

,Authors,Author(s)_ID,Authors_Rev,Author(s)_ID_Rev,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,Page_end,Page_count,Cited_by,DOI,Link,Affiliations,Authors_with_affiliations,Abstract,Author_Keywords,Index_Keywords,ISSN,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID,Title,Cited_by,Years,cant_coautores,citation_journal
0,Vanzolini P.E.,6602373241,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,611,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Museu de Zoologia, Universidade de São Paulo, ...","Vanzolini, P.E., Museu de Zoologia, Universida...","Gymnodactylus amarali Barbour, 1925, was previ...",Lizards; Speciation; Systematics,animal; article; Brazil; classification; compa...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573,"On Gymnodactylus amarali Barbour, 1925, with t...",7.0,14.0,1.0,9.146637
1,De Wagener A.L.R.,7004422261,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,623,NaN,6.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Departamento de Química, Pontifícia Universida...","De Wagener, A.L.R., Departamento de Química, P...",This paper addresses the limitations the scarc...,Coastal management; Developing countries; Envi...,article; developing country; environmental mon...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222,Constraints to the implementation of effective...,6.0,14.0,1.0,9.146637
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Mexias A.S.,11739855600,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,Geochemical modeling of gold precipitation con...,7.0,14.0,7.0,9.146637


In [43]:
# Revisión de dimensiones del df original con el nuevo df
print(df_consol.shape, df.shape)

(177350, 35) (177350, 30)


In [44]:
# Unión del df consolidado con el df autores
df_consol = pd.concat([df_consol, df_authors], axis=1, sort=False) #df_consol.merge(df_authors, left_on='Authors_Rev', right_on='Authors_Rev')

In [45]:
# Revisión df consolidado
df_consol.head(3)

,Authors,Author(s)_ID,Authors_Rev,Author(s)_ID_Rev,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,Page_end,Page_count,Cited_by,DOI,Link,Affiliations,Authors_with_affiliations,Abstract,Author_Keywords,Index_Keywords,ISSN,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID,Title,Cited_by,Years,cant_coautores,citation_journal,Authors_Rev,Author(s)_ID_Rev,Total_Citation,Total_Title,Max_Citation,Total_Journal,Delta_publications,autor_career,citation_journal,Delta_coautores
0,Vanzolini P.E.,6602373241,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,611,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Museu de Zoologia, Universidade de São Paulo, ...","Vanzolini, P.E., Museu de Zoologia, Universida...","Gymnodactylus amarali Barbour, 1925, was previ...",Lizards; Speciation; Systematics,animal; article; Brazil; classification; compa...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573,"On Gymnodactylus amarali Barbour, 1925, with t...",7.0,14.0,1.0,9.146637,Vanzolini P.E.,6602373241,62.0,5.0,17.0,1.0,0.0,3.0,9.146637,NaN
1,De Wagener A.L.R.,7004422261,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,623,NaN,6.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Departamento de Química, Pontifícia Universida...","De Wagener, A.L.R., Departamento de Química, P...",This paper addresses the limitations the scarc...,Coastal management; Developing countries; Envi...,article; developing country; environmental mon...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222,Constraints to the implementation of effective...,6.0,14.0,1.0,9.146637,De Wagener A.L.R.,7004422261,6.0,1.0,6.0,1.0,0.0,0.0,9.146637,NaN
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Mexias A.S.,11739855600,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,Geochemical modeling of gold precipitation con...,7.0,14.0,7.0,9.146637,Mexias A.S.,11739855600,11.0,2.0,7.0,1.0,0.0,2.0,9.146637,NaN


In [46]:
# Revisión de dimensiones del df consolidad con el nuevo df de autores
print(df_consol.shape,df.shape)

(177350, 45) (177350, 30)


In [47]:
# Lista para la revisión de columnas duplicadas por las uniones
lista = [df_consol.columns]

In [48]:
lista

[Index(['Authors', 'Author(s)_ID', 'Authors_Rev', 'Author(s)_ID_Rev', 'Title',
        'Year', 'Source_title', 'Volume', 'Issue', 'Art._No.', 'Page_start',
        'Page_end', 'Page_count', 'Cited_by', 'DOI', 'Link', 'Affiliations',
        'Authors_with_affiliations', 'Abstract', 'Author_Keywords',
        'Index_Keywords', 'ISSN', 'ISBN', 'CODEN', 'PubMed_ID', 'Document_Type',
        'Publication_Stage', 'Access_Type', 'Source', 'EID', 'Title',
        'Cited_by', 'Years', 'cant_coautores', 'citation_journal',
        'Authors_Rev', 'Author(s)_ID_Rev', 'Total_Citation', 'Total_Title',
        'Max_Citation', 'Total_Journal', 'Delta_publications', 'autor_career',
        'citation_journal', 'Delta_coautores'],
       dtype='object')]

In [49]:
# Eliminación de las columnas duplicadas
df_consol_proc = df_consol.loc[:, ~df_consol.columns.duplicated()]

In [53]:
# Eliminación de los articulos que no tienen autores
df_consol_proc = df_consol_proc[pd.notnull(df_consol_proc['Authors_Rev'])]

In [54]:
# Revisión de las dimensiones después de borrar las columnas duplicadas
df_consol_proc.shape

(177350, 40)

In [55]:
# Revisión del df consolidado después de borrar duplicados
df_consol_proc.head()

,Authors,Author(s)_ID,Authors_Rev,Author(s)_ID_Rev,Title,Year,Source_title,Volume,Issue,Art._No.,Page_start,Page_end,Page_count,Cited_by,DOI,Link,Affiliations,Authors_with_affiliations,Abstract,Author_Keywords,Index_Keywords,ISSN,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID,Years,cant_coautores,citation_journal,Total_Citation,Total_Title,Max_Citation,Total_Journal,Delta_publications,autor_career,Delta_coautores
0,Vanzolini P.E.,6602373241,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,611,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Museu de Zoologia, Universidade de São Paulo, ...","Vanzolini, P.E., Museu de Zoologia, Universida...","Gymnodactylus amarali Barbour, 1925, was previ...",Lizards; Speciation; Systematics,animal; article; Brazil; classification; compa...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573,14.0,1.0,9.146637,62.0,5.0,17.0,1.0,0.0,3.0,NaN
1,De Wagener A.L.R.,7004422261,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,613,623,NaN,6.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Departamento de Química, Pontifícia Universida...","De Wagener, A.L.R., Departamento de Química, P...",This paper addresses the limitations the scarc...,Coastal management; Developing countries; Envi...,article; developing country; environmental mon...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222,14.0,1.0,9.146637,6.0,1.0,6.0,1.0,0.0,0.0,NaN
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Mexias A.S.,11739855600,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,14.0,7.0,9.146637,11.0,2.0,7.0,1.0,0.0,2.0,NaN
3,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Berger G.,7202349172,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,14.0,1.0,9.146637,7.0,1.0,7.0,1.0,0.0,0.0,NaN
4,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Gomes M.E.B.,11738879800,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,717,728,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Universidade Federal do Rio Grande do Sul, Ins...","Mexias, A.S., Universidade Federal do Rio Gran...",A geochemical modeling of gold deposition was ...,Fluid-rock interaction; Geochemical modeling; ...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319,14.0,1.0,9.146637,11.0,2.0,7.0,1.0,0.0,2.0,NaN


In [57]:
df_consol_proc.to_csv('scopus_con_indicadores.csv')